In [1]:
# import sys
# !{sys.executable} -m pip install python_dotenv
# !{sys.executable} -m pip install python_binance
import os
from binance.client import Client
from binance.spot import Spot

import pandas as pd
from dotenv import load_dotenv


In [2]:
# read from local .env file
load_dotenv() 

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_API_SECRET']

client = Client()
spotClient = Spot()
# myClient = Client(api_key, api_secret, testnet=True)

In [3]:
# INPUT
## d2, d3, d4
timeframe = "3d"
## ema (20 or 25)
ema_size = 25
## A symbol with streak >= this will be printed out
streak_threshold = 44
quote_asset = "BTC"


In [4]:
# Function to query Binance for candlestick data
def get_candlestick_data(symbol, timeframe, qty):
    # Retrieve the raw data
    raw_data = spotClient.klines(symbol=symbol, interval=timeframe, limit=qty)
    # Set up the return array
    converted_data = []
    # Convert each element into a Python dictionary object, then add to converted_data
    for candle in raw_data:
        # Dictionary object
        converted_candle = {
            'time': candle[0],
            'open': float(candle[1]),
            'high': float(candle[2]),
            'low': float(candle[3]),
            'close': float(candle[4]),
            'volume': float(candle[5]),
            'close_time': candle[6],
            'quote_asset_volume': float(candle[7]),
            'number_of_trades': int(candle[8]),
            'taker_buy_base_asset_volume': float(candle[9]),
            'taker_buy_quote_asset_volume': float(candle[10])
        }
        # Add to converted_data
        converted_data.append(converted_candle)
    # Return converted data
    return converted_data

In [5]:
# Define function to calculate an arbitrary EMA    
def calc_generic_ema(symbol, timeframe, ema_size):
    raw_data = get_candlestick_data(symbol=symbol, timeframe=timeframe, qty=1000)
    # Convert into Dataframe
    dataframe = pd.DataFrame(raw_data)
    # Create column string
    ema_name = "ema_" + str(ema_size)
    # Create the multiplier
    multiplier = 2/(ema_size + 1)
    # Calculate the initial value (SMA)
    # pd.set_option('display.max_columns', None) # <- use this to show all columns
    # pd.set_option('display.max_rows', None) # <- use this to show all the rows
    initial_mean = dataframe['close'].head(ema_size).mean()

    # Iterate through Dataframe
    for i in range(len(dataframe)):
        if i == ema_size:
            dataframe.loc[i, ema_name] = initial_mean
        elif i > ema_size:
            ema_value = dataframe.loc[i, 'close'] * multiplier + dataframe.loc[i-1, ema_name]*(1-multiplier)
            dataframe.loc[i, ema_name] = ema_value
        else:
            dataframe.loc[i, ema_name] = 0.00
    # print(dataframe) # <- use this to print the dataframe if you want to inspect
    return dataframe

In [6]:
# GET SYMBOL LIST
info = client.get_exchange_info()
coinlist = []
for c in info['symbols']:
    if c['quoteAsset']==quote_asset and c['status']=="TRADING":
        coinlist.append(c['symbol'])
        
print(len(coinlist))
# sublist = coinlist[0::10]
sublist = coinlist
print('Scanning ' + str(len(sublist)) + ' trading pairs...')

282
Scanning 282 trading pairs...


In [7]:
# Progress bar
from ipywidgets import IntProgress
from IPython.display import display
import time

max_count = len(sublist)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

count = 0
# MAIN
for idx, symbol in enumerate(sublist):
    # update progress bar
    f.value += 1
    
    df = calc_generic_ema(symbol, timeframe, ema_size)

    df['time'] = pd.to_datetime(df['time'], unit='ms')

    del df['volume']
    del df['close_time']
    del df['quote_asset_volume']
    del df['number_of_trades']
    del df['taker_buy_base_asset_volume']
    del df['taker_buy_quote_asset_volume']

    df['oversell_day'] = 0
    df.loc[df['high'] > df['ema_25'], 'oversell_day'] = 1
    df.loc[df['ema_25'] == 0, 'oversell_day'] = 0


    df['streak'] = df.groupby((df["oversell_day"] != df["oversell_day"].shift()).cumsum()).cumcount()+1
    df.loc[df['oversell_day'] == 0, 'streak'] = 0
    
    if (df['streak'].iloc[-1] > streak_threshold):
        print('Symbol: '+ symbol + ' is having an oversell streak')
        display(df['streak'].iloc[-1])
        display(df.tail(30))
        
    if f.value == max_count:
        print('Scan Completed!')

IntProgress(value=0, max=282)

Symbol: VIBBTC is having an oversell streak


93

,time,open,high,low,close,ema_25,oversell_day,streak
624,2022-12-15,0.000005,0.000005,0.000004,0.000005,0.000004,1,64
625,2022-12-18,0.000004,0.000005,0.000004,0.000004,0.000004,1,65
626,2022-12-21,0.000004,0.000004,0.000004,0.000004,0.000004,1,66
627,2022-12-24,0.000004,0.000004,0.000004,0.000004,0.000004,1,67
628,2022-12-27,0.000004,0.000004,0.000004,0.000004,0.000004,1,68
629,2022-12-30,0.000004,0.000004,0.000004,0.000004,0.000004,1,69
630,2023-01-02,0.000004,0.000004,0.000004,0.000004,0.000004,1,70
631,2023-01-05,0.000004,0.000005,0.000004,0.000004,0.000004,1,71
632,2023-01-08,0.000004,0.000004,0.000004,0.000004,0.000004,1,72
633,2023-01-11,0.000004,0.000005,0.000004,0.000004,0.000004,1,73


Symbol: OPBTC is having an oversell streak


48

,time,open,high,low,close,ema_25,oversell_day,streak
66,2022-12-16,0.000061,0.000062,0.000054,0.000055,0.000055,1,19
67,2022-12-19,0.000055,0.000058,0.000053,0.000057,0.000055,1,20
68,2022-12-22,0.000057,0.000057,0.000054,0.000055,0.000055,1,21
69,2022-12-25,0.000055,0.000057,0.000054,0.000056,0.000055,1,22
70,2022-12-28,0.000056,0.000056,0.000053,0.000055,0.000055,1,23
71,2022-12-31,0.000055,0.000058,0.000055,0.000057,0.000055,1,24
72,2023-01-03,0.000057,0.000063,0.000057,0.000061,0.000056,1,25
73,2023-01-06,0.000061,0.000075,0.000059,0.000074,0.000057,1,26
74,2023-01-09,0.000074,0.000079,0.000069,0.000071,0.000058,1,27
75,2023-01-12,0.000071,0.000086,0.000068,0.000082,0.000060,1,28


Scan Completed!
